In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

Processing...
Done!


In [3]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [4]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [5]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [6]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
for epoch in range(1, 10):
    train(epoch)
    test()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.286185
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.297168
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.292522
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.273262
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.259386
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.235476
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.215951
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.177022
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.087188
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.090649
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.946579
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.622543
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.594562
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.228907
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.010023
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.900797
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.820880
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.711075
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.747931
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.726379
Train Epoch: 1 [12800/60000 (

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  if __name__ == '__main__':



Test set: Average loss: 0.1719, Accuracy: 9498/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.144047
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.375750
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.351152
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.359791
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.128688
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.185730
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.139461
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.107400
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.208529
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.111886
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.312961
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.184015
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.177168
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.118435
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.400152
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.106143
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.266197
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.162897
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.188851
Train Epoch: 2 [12

Train Epoch: 3 [43520/60000 (72%)]	Loss: 0.197992
Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.067133
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.158604
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.043506
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.050892
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.060524
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.092141
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.115846
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.222170
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.008818
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.037054
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.111147
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.088440
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.153818
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.130907
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.077996
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.026025
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.293833
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.039686
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.065265


Train Epoch: 5 [26880/60000 (45%)]	Loss: 0.146278
Train Epoch: 5 [27520/60000 (46%)]	Loss: 0.055024
Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.023642
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.085674
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.111739
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.122900
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.188082
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.038924
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.030563
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.134462
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.048400
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.066544
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.137128
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.236238
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.082078
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.068759
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.117320
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.060915
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.079128
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.082681


Train Epoch: 7 [10240/60000 (17%)]	Loss: 0.077979
Train Epoch: 7 [10880/60000 (18%)]	Loss: 0.079700
Train Epoch: 7 [11520/60000 (19%)]	Loss: 0.037119
Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.128602
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.011901
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.067379
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.093478
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.048450
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.035686
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.061650
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.051387
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.287052
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.022693
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.013786
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.057168
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.096905
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.039259
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.022119
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.031344
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.045990


Train Epoch: 8 [54400/60000 (91%)]	Loss: 0.120806
Train Epoch: 8 [55040/60000 (92%)]	Loss: 0.073733
Train Epoch: 8 [55680/60000 (93%)]	Loss: 0.015356
Train Epoch: 8 [56320/60000 (94%)]	Loss: 0.007672
Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.055976
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.031533
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.044119
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.032433
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.046899

Test set: Average loss: 0.0490, Accuracy: 9844/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.042370
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.137671
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.061587
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.099699
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.158568
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.062314
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.047225
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.125855
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.035466
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.029024
Train Epoch: